In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
train_dataset = pd.read_csv('/content/PM_train.txt',sep=' ')
train_dataset.drop(train_dataset.columns[[26,27]], axis=1,inplace=True)
col_name = ['id','cycle','set1','set2','set3','s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14','s14','s15','s16','s17','s18','s19','s20']
train_dataset.columns = col_name
print(train_dataset.head())

   id  cycle    set1    set2   set3      s1      s2       s3       s4     s5  \
0   1      2  0.0019 -0.0003  100.0  518.67  642.15  1591.82  1403.14  14.62   
1   1      3 -0.0043  0.0003  100.0  518.67  642.35  1587.99  1404.20  14.62   
2   1      4  0.0007  0.0000  100.0  518.67  642.35  1582.79  1401.87  14.62   
3   1      5 -0.0019 -0.0002  100.0  518.67  642.37  1582.85  1406.22  14.62   
4   1      6 -0.0043 -0.0001  100.0  518.67  642.10  1584.47  1398.37  14.62   

   ...     s12      s13      s14     s14   s15  s16   s17    s18    s19  \
0  ...  522.28  2388.07  8131.49  8.4318  0.03  392  2388  100.0  39.00   
1  ...  522.42  2388.03  8133.23  8.4178  0.03  390  2388  100.0  38.95   
2  ...  522.86  2388.08  8133.83  8.3682  0.03  392  2388  100.0  38.88   
3  ...  522.19  2388.04  8133.80  8.4294  0.03  393  2388  100.0  38.90   
4  ...  521.68  2388.03  8132.85  8.4108  0.03  391  2388  100.0  38.98   

       s20  
0  23.4236  
1  23.3442  
2  23.3739  
3  23.4044  
4  

In [ ]:
test_dataset = pd.read_csv('/content/PM_test.txt',sep=' ')
test_dataset.drop(test_dataset.columns[[26,27]], axis=1,inplace=True)
test_dataset.columns = col_name
print(test_dataset.head())

   id  cycle    set1    set2   set3      s1      s2       s3       s4     s5  \
0   1      2 -0.0027 -0.0003  100.0  518.67  641.71  1588.45  1395.42  14.62   
1   1      3  0.0003  0.0001  100.0  518.67  642.46  1586.94  1401.34  14.62   
2   1      4  0.0042  0.0000  100.0  518.67  642.44  1584.12  1406.42  14.62   
3   1      5  0.0014  0.0000  100.0  518.67  642.51  1587.19  1401.92  14.62   
4   1      6  0.0012  0.0003  100.0  518.67  642.11  1579.12  1395.13  14.62   

   ...     s12      s13      s14     s14   s15    s16     s17    s18    s19  \
0  ...  522.16  2388.06  8139.62  8.3803  0.03  393.0  2388.0  100.0  39.02   
1  ...  521.97  2388.03  8130.10  8.4441  0.03  393.0  2388.0  100.0  39.08   
2  ...  521.38  2388.05  8132.90  8.3917  0.03  391.0  2388.0  100.0  39.00   
3  ...  522.15  2388.03  8129.54  8.4031  0.03  390.0  2388.0  100.0  38.99   
4  ...  521.92  2388.08  8127.46  8.4238  0.03  392.0  2388.0  100.0  38.91   

       s20  
0  23.3916  
1  23.4166  
2  23

In [ ]:
truth_ds = pd.read_csv('/content/PM_truth.txt',sep=' ')
truth_ds.drop(truth_ds.columns[[1]], axis=1,inplace=True)
truth_ds.columns = ['more']
truth_ds['id'] = truth_ds.index+1
print(truth_ds.head())

   more  id
0    98   1
1    69   2
2    82   3
3    91   4
4    93   5


In [ ]:
rul=pd.DataFrame(test_dataset.groupby("id")['cycle'].max()).reset_index()
rul.columns = ['id','max']
rul.head()

,id,max
0,1,31
1,2,49
2,3,126
3,4,106
4,5,98


In [ ]:
truth_ds['rtf']=truth_ds['more']+rul["max"]
truth_ds.head ()

,more,id,rtf
0,98,1,129.0
1,69,2,118.0
2,82,3,208.0
3,91,4,197.0
4,93,5,191.0


In [ ]:
truth_ds.drop('more',axis=1,inplace=True)
test_dataset=test_dataset.merge(truth_ds, on= ['id'], how= "left")
test_dataset[ 'ttf']=test_dataset['rtf'] - test_dataset['cycle']
test_dataset.drop ('rtf', axis=1, inplace=True)
test_dataset. head()

,id,cycle,set1,set2,set3,s1,s2,s3,s4,s5,...,s13,s14,s14,s15,s16,s17,s18,s19,s20,ttf
0,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,2388.06,8139.62,8.3803,0.03,393.0,2388.0,100.0,39.02,23.3916,127.0
1,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,2388.03,8130.10,8.4441,0.03,393.0,2388.0,100.0,39.08,23.4166,126.0
2,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,2388.05,8132.90,8.3917,0.03,391.0,2388.0,100.0,39.00,23.3737,125.0
3,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,2388.03,8129.54,8.4031,0.03,390.0,2388.0,100.0,38.99,23.4130,124.0
4,1,6,0.0012,0.0003,100.0,518.67,642.11,1579.12,1395.13,14.62,...,2388.08,8127.46,8.4238,0.03,392.0,2388.0,100.0,38.91,23.3467,123.0


In [ ]:
train_dataset[ 'ttf'] = train_dataset.groupby(["id"])['cycle'].transform(max)-train_dataset['cycle']
train_dataset.head()

,id,cycle,set1,set2,set3,s1,s2,s3,s4,s5,...,s13,s14,s14,s15,s16,s17,s18,s19,s20,ttf
0,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
1,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
2,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
3,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187
4,1,6,-0.0043,-0.0001,100.0,518.67,642.10,1584.47,1398.37,14.62,...,2388.03,8132.85,8.4108,0.03,391,2388,100.0,38.98,23.3669,186


In [ ]:
df_train = train_dataset.copy()
df_test = test_dataset.copy()
period = 30
df_train['label_bc'] = df_train ["ttf"].apply(lambda x: 1 if x <= period else 0)
df_test['label_bc'] = df_test ['ttf'].apply(lambda x: 1 if x <= period else 0)
df_test = df_test.dropna()
df_train = df_train.dropna()
df_train.head()

,id,cycle,set1,set2,set3,s1,s2,s3,s4,s5,...,s14,s14,s15,s16,s17,s18,s19,s20,ttf,label_bc
0,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190,0
1,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189,0
2,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188,0
3,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187,0
4,1,6,-0.0043,-0.0001,100.0,518.67,642.10,1584.47,1398.37,14.62,...,8132.85,8.4108,0.03,391,2388,100.0,38.98,23.3669,186,0


In [ ]:
x_train = df_train.iloc[ : , : -1].values
y_train = df_train. iloc[: , -1:].values

In [ ]:
y_train.shape

(20630, 1)

In [ ]:
model = LogisticRegression()
model = model.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
x_test = df_test.iloc[ : , : -1].values
y_test = df_test. iloc[: , -1:].values

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
import sklearn.metrics as metrics
mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(y_test,y_pred)
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

MAE: 0.0020943285582642203
MSE: 0.0020943285582642203
RMSE: 0.045763834610576724
R-Squared: 0.919966176205556


In [ ]:
accuracy_score(y_test,y_pred)

0.9979056714417358

In [ ]:
from sklearn import tree
decision = tree.DecisionTreeClassifier(criterion="gini")
decision.fit(x_train,y_train)

DecisionTreeClassifier()

In [ ]:
y_pred1 = decision.predict(x_test)

In [ ]:
from sklearn import svm
support = svm.SVC()
support.fit(x_train,y_train)
y_pred2 = support.predict(x_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neighbors = KNeighborsClassifier(n_neighbors=5)
neighbors.fit(x_train,y_train)
y_pred3 = neighbors.predict(x_test)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
from sklearn.model_selection import cross_val_score
logistic = cross_val_score(model, x_train, y_train, scoring='r2', cv=5)
dec = cross_val_score(decision, x_train, y_train, scoring='r2', cv=5)
support_vec = cross_val_score(decision, x_train, y_train, scoring='r2', cv=5)
knn = cross_val_score(neighbors, x_train, y_train, scoring='r2', cv=5)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

Logistic Regression: [1.         0.9867131  1.         0.99240749 0.98481497]
Decision tree: [1. 1. 1. 1. 1.]
Support Vector Machine: [1. 1. 1. 1. 1.]
KNN: [0.6944013  0.80639089 0.72287323 0.69629943 0.67352189]


In [ ]:
from sklearn.model_selection import cross_val_score
logistic = cross_val_score(model, x_train, y_train, scoring='r2', cv=5)
dec = cross_val_score(decision, x_train, y_train, scoring='r2', cv=5)
support_vec = cross_val_score(decision, x_train, y_train, scoring='r2', cv=5)
knn = cross_val_score(neighbors, x_train, y_train, scoring='r2', cv=5)
print("Logistic Regression:",logistic)
print("Decision tree:",dec)
print("Support Vector Machine:",support_vec)
print("KNN:",knn)

Logistic Regression: [1.         0.9867131  1.         0.99240749 0.98481497]
Decision tree: [1. 1. 1. 1. 1.]
Support Vector Machine: [1. 1. 1. 1. 1.]
KNN: [0.6944013  0.80639089 0.72287323 0.69629943 0.67352189]


In [ ]:
import joblib
joblib.dump(model,'/content/engine_model.pkl')

['/content/engine_model.pkl']

In [ ]:
from copyreg import pickle
from io import StringIO
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import pickle

In [ ]:
model=joblib.load('/content/engine_model.pkl')

In [ ]:
def predict(data):
    try:
        col_name = ['id','cycle','set1','set2','set3','s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14','s14','s15','s16','s17','s18','s19','s20']
        test_dataset = pd.DataFrame([data],columns=col_name)
        rul=pd.DataFrame(test_dataset.groupby("id")['cycle'].max()).reset_index()
        rul.columns = ['id','max']
        truth_ds['rtf']=truth_ds['more']+rul["max"]
        truth_ds.head()
        truth_ds['rtf']=truth_ds['more']+rul["max"]
        test_dataset=test_dataset.merge(truth_ds, on= ['id'], how= "left")
        test_dataset[ 'ttf']=test_dataset['rtf'] - test_dataset['cycle']
        test_dataset.drop ('rtf', axis=1, inplace=True)
        df_test = test_dataset.copy()
        period = 30
        df_test['label_bc'] = df_test ['ttf'].apply(lambda x: 1 if x <= period else 0)
        df_test = df_test.dropna()
        if len(df_test.index) == 0 :
            return True
        x_test = df_test.iloc[ : , : -1].values
        y_pred = model.predict(x_test)
        return True if y_pred[0] else False
    except:
        return True